# Read data

In [23]:
# import zipfile

# with zipfile.ZipFile('dm2021-lab2-hw2.zip', 'r') as zip_ref:
#     zip_ref.extractall('./kaggle_data')

In [1]:
import pandas as pd

df_tweets = pd.read_json("./kaggle_data/tweets_DM.json", lines=True)
df_tot = pd.read_csv("./kaggle_data/data_identification.csv")
df_emotion = pd.read_csv("./kaggle_data/emotion.csv")

# Observe dataset

In [2]:
# df_tweets
print (type(df_tweets))
print (df_tweets.columns)
print (type(df_tweets['_source']), df_tweets['_source'].shape)
print (type(df_tweets['_source'][0]), df_tweets['_source'][0].keys())
print (type(df_tweets['_source'][0]['tweet']), df_tweets['_source'][0]['tweet'].keys())
print (df_tweets['_source'][0]['tweet'])
print ()

# df_tot (train or test)
print (df_tot.columns)
print ()

# df_emotion
print (df_emotion.columns)

<class 'pandas.core.frame.DataFrame'>
Index(['_score', '_index', '_source', '_crawldate', '_type'], dtype='object')
<class 'pandas.core.series.Series'> (1867535,)
<class 'dict'> dict_keys(['tweet'])
<class 'dict'> dict_keys(['hashtags', 'tweet_id', 'text'])
{'hashtags': ['Snapchat'], 'tweet_id': '0x376b20', 'text': 'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'}

Index(['tweet_id', 'identification'], dtype='object')

Index(['tweet_id', 'emotion'], dtype='object')


In [3]:
df_emotion.groupby('emotion').count()

,tweet_id
emotion,
anger,39867
anticipation,248935
disgust,139101
fear,63999
joy,516017
sadness,193437
surprise,48729
trust,205478


In [4]:
print ("There are totally", len(df_tweets['_source']), "tweets")
df_tweets['_source'][0]['tweet']

There are totally 1867535 tweets


{'hashtags': ['Snapchat'],
 'tweet_id': '0x376b20',
 'text': 'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'}

In [ ]:
ids = []
hashtags = []
tweets = []
# for i in range(df_tweets.shape[0]):
for i in range(100):
    twid = df_tweets['_source'][i]['tweet']['tweet_id']
    tot = df_tot.loc[df_tot['tweet_id']==twid]
    if tot['identification'].to_list()[-1]=='train':
#         emo = df_emotion.loc[df_emotion['tweet_id']==twid]['emotion']
#         print (emo, (df_tweets['_source'][i]['tweet']['hashtags']))
        print (df_tweets['_source'][i]['tweet']['text'])
        print ()

# make data_dict and emoji_list

data_dict \[ tweet_id \] = \[ text, hashtags, emotion\ ]

In [6]:
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
import emoji
import pickle

reg_tokenizer = RegexpTokenizer("\w+[\.|\']\w+|\w+|\#\w+|\@\w+|\u2764|[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F1E0-\U0001F1FF]")


def extract_emojis(s):
    return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

emoji_set = set()
data_dict = {}
for i in range(len(df_tweets)):
    text = df_tweets['_source'][i]['tweet']['text']
    text = text.replace('<LH>', '')
    emoji_set = emoji_set.union(set(extract_emojis(text)))
    tid = df_tweets['_source'][i]['tweet']['tweet_id']
    tags = df_tweets['_source'][i]['tweet']['hashtags']
    data_dict[tid] = [text, tags, ''] # [text, hashtags, emo]

    # print (text)
#     text_list_small.append(text)
#     id_list_small.append(tid)
#     text = tokenizer.tokenize(text)

emoji_set = list(emoji_set)
pickle.dump(emoji_set, open('emoji_list.pkl', 'wb'))


In [8]:
data_dict['0x376b20']

['People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s ',
 ['Snapchat'],
 '']

# make data table

In [9]:
for _, row in df_emotion.iterrows():
    if row.tweet_id not in data_dict:
        print (row.tweet_id)
    data_dict[row.tweet_id][-1] = row.emotion
    
    
df_data = pd.DataFrame.from_dict(data_dict, orient='index', columns=['text', 'hashtags', 'emotion'])
df_data['ID'] = df_data.index
df_data = df_data[['ID', 'text', 'hashtags', 'emotion']]



In [10]:
df_data.head(5)

,ID,text,hashtags,emotion
0x376b20,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],anticipation
0x2d5350,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",sadness
0x28b412,0x28b412,"Confident of your obedience, I write to you, k...",[bibleverse],
0x1cd5b0,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂,[],fear
0x2de201,0x2de201,"""Trust is not the same as faith. A friend is s...",[],


In [12]:
## make training dataset
df_data.loc[df_data['emotion']!=''].to_csv('train_w_hashtag.tsv', sep='\t', index=False)
df_data.loc[df_data['emotion']!=''].head(5)

,ID,text,hashtags,emotion
0x376b20,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],anticipation
0x2d5350,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",sadness
0x1cd5b0,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂,[],fear
0x1d755c,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",joy
0x2c91a8,0x2c91a8,Still waiting on those supplies Liscus.,[],anticipation


In [14]:
## make testing dataset
df_data.loc[df_data['emotion']==''].to_csv('test.tsv', sep='\t', index=False)
df_data.loc[df_data['emotion']==''].head(3)

,ID,text,hashtags,emotion
0x28b412,0x28b412,"Confident of your obedience, I write to you, k...",[bibleverse],
0x2de201,0x2de201,"""Trust is not the same as faith. A friend is s...",[],
0x218443,0x218443,When do you have enough ? When are you satisfi...,"[materialism, money, possessions]",


# tokenize and vectorize

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

reg_tokenizer = RegexpTokenizer("\w+[\.|\']\w+|\w+|\#\w+|\@\w+|\u2764|[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F1E0-\U0001F1FF]")
tf_idf = TfidfVectorizer(max_features=1000, tokenizer=reg_tokenizer.tokenize)
tf_idf.fit(df_data['text'])

# for a classificaiton problem, you need to provide both training & testing data
X = tf_idf.transform(df_data.loc[df_data['emotion']!=""]['text'])
y = df_data.loc[df_data['emotion']!=""]['emotion']

## take a look at data dimension :)
print('X_train.shape: ', X.shape)
print('y_train.shape: ', y.shape)

/home/lihsuan/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


X_train.shape:  (1455563, 1000)
y_train.shape:  (1455563,)


# decomposition with SVD

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=800, n_iter=200, random_state=42)
X = svd.fit_transform(X)
X.shape

# confusion matrix plotting module

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix',
                          cmap=sns.cubehelix_palette(as_cmap=True)):

    classes.sort()
    tick_marks = np.arange(len(classes))    
    
    fig, ax = plt.subplots(figsize=(5,5))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels = classes,
           yticklabels = classes,
           title = title,
           xlabel = 'True label',
           ylabel = 'Predicted label')

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    ylim_top = len(classes) - 0.5
    plt.ylim([ylim_top, -.5])
    plt.tight_layout()
    plt.show()

# 1st model: Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0)

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))
print('testing accuracy: {}'.format(round(acc_test, 2)))

DT_cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(classification_report(y_true=y_test, y_pred=y_test_pred))

In [ ]:
my_tags = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']
plot_confusion_matrix(DT_cm, classes=my_tags, title='Confusion matrix')

# 2nd model: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
RF_model = RandomForestClassifier(max_depth=200)
RF_model = RF_model.fit(X_train, y_train)

## predict!
y_train_pred = RF_model.predict(X_train)
y_test_pred = RF_model.predict(X_test)

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))
print('testing accuracy: {}'.format(round(acc_test, 2)))

RF_cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(classification_report(y_true=y_test, y_pred=y_test_pred))

In [ ]:
my_tags = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']
plot_confusion_matrix(RF_cm, classes=my_tags, title='Confusion matrix')

# 3rd model: Bert

In [3]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output
from transformers import BertForMaskedLM
import pandas as pd
from torch.utils.data import Dataset
import pickle

In [40]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
tokenizer.add_tokens(emoji_set)
    
class Dataset(Dataset):
    ## read the preprocessed .tsv and  initailize some parameter
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]
        self.mode = mode
        self.df = pd.read_csv(mode + "_w_hashtag.tsv", sep="\t", lineterminator='\n').fillna("")
        self.len = len(self.df)
        self.label_map = {'anger':0, 'anticipation':1, 'disgust':2, 'fear':3, 'joy':4, 'sadness':5, 'surprise':6, 'trust':7}
        print (self.df.emotion.value_counts() / len(self.df))
        tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
        tokenizer.add_tokens(emoji_set)
        ## will use bert tokenizer
        self.tokenizer = tokenizer
    
    
    ## return a training / testing data
    def __getitem__(self, idx):
        if self.mode == "test":
            text_id, text_a, text_hashtag = self.df.iloc[idx, :3].values
            label_tensor = None
        else:
            text_id, text_a, text_hashtag, label = self.df.iloc[idx, :].values
            
            ## convert the label from text to index
            ## in order for making tensor
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)
            
        ## create BERT token for a tweet
        ## add [SEP]
        word_pieces = ["[CLS]"]
        # tokens_a = self.tokenizer.tokenize(text_a)
        text_a = text_a + "".join(text_hashtag)
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)
        
        ## convert tokens sequence to index sequence
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        segments_tensor = torch.tensor([0] * len_a, dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    def __len__(self):
        return self.len
    
    
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = Dataset("train", tokenizer=tokenizer)
# testset = Dataset("test", tokenizer=tokenizer)



    1.0
Name: emotion, dtype: float64


train set:
joy             0.354514
anticipation    0.171023
trust           0.141167
sadness         0.132895
disgust         0.095565
fear            0.043969
surprise        0.033478
anger           0.027389

In [37]:
# 1366
trainset.df.iloc[1366]["text"]

"You don't choose your  😱 but you get to choose your #fancy\r#FancyVsFear"

In [38]:
df_emotion.loc[df_emotion['tweet_id']=='0x1d69d5']
df_data.loc[df_data['ID']=='0x1d69d5']

,ID,text,hashtags,emotion
0x1d69d5,0x1d69d5,You don't choose your 😱 but you get to choose...,"[fancy, FancyVsFear]",fear


In [41]:
def see_set(t_set):
    for i in range(100):
        # 選擇第一個樣本
        sample_idx = i

        # 將原始文本拿出做比較
        text_id, text_a, hashtags, label = t_set.df.iloc[sample_idx].values

        # 利用剛剛建立的 Dataset 取出轉換後的 id tensors
        tokens_tensor, segments_tensor, label_tensor = t_set[sample_idx]

        # 將 tokens_tensor 還原成文本
        tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
        combined_text = " ".join(tokens)

        # 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
        print(f"""[原始文本]
        句子 1：{text_a}
        分類  ：{label}

        --------------------

        [Dataset 回傳的 tensors]
        tokens_tensor  ：{tokens_tensor}

        segments_tensor：{segments_tensor}

        label_tensor   ：{label_tensor}

        --------------------

        [還原 tokens_tensors]
        {combined_text}
        """)

see_set(testset)

[原始文本]
        句子 1：Confident of your obedience, I write to you, knowing that you will do even more than I ask. (Philemon 1:21) 3/4 #bibleverse  
        分類  ：

        --------------------

        [Dataset 回傳的 tensors]
        tokens_tensor  ：tensor([  101, 10173, 14810, 32997, 10108, 12787, 34853, 42571, 10421,   117,
          151, 26039, 10114, 10855,   117, 72161, 10203, 10855, 11229, 10154,
        12818, 10772, 10948,   151, 30316,   119,   113, 17804, 81258,   122,
          131, 10259,   114,   124,   120,   125,   108, 24276, 30340,   138,
          112, 24276, 30340,   112,   140,   102])

        segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

        label_tensor   ：None

        --------------------

        [還原 tokens_tensors]
        [CLS] con ##fi ##dent of your obe ##dien ##ce , i write to you , knowing that you will do even more than i ask . ( 

In [76]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `Dataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `Dataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors


def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    # 測試集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids


# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 4
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

# testloader = DataLoader(testset, batch_size=BATCH_SIZE, 
#                          collate_fn=create_mini_batch)

In [58]:
data = next(iter(trainloader))
# data = next(iter(testloader))

tokens_tensors, segments_tensors, \
    masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([4, 66]) 
tensor([[   101,  11227,  10488,  10404,    107,  33989,  10525,  10125,    108,
          52539,  64415,  20452,    107,  14650,  10346,  10102,  84202,  33492,
            119,  10707,  10311,  10564,    119,    119,    119,    119,  10203,
            112,    161,    138,    112,    161,    156,    143,    158,    145,
            150,    143,    162,    112,    140,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0],
        [   101,    137,  14319,  22192,  10416,  10146,  11312,  11811,    117,
          29104,  10127,  30728,  10114,    108,  12487,  44885,  12096,  10103,
          10228,    119,  11523,    143,    108,  29104,  33667,  12207,    119,
            108,  30047,    138,    112,    148,    160,    147,    147,    158,
            160,    147,    

In [77]:
# 載入一個可以做中文多分類任務的模型，n_class = 3
from transformers import BertForSequenceClassification

# PRETRAINED_MODEL_NAME = 'bert-base-uncased'
PRETRAINED_MODEL_NAME = 'bert-base-multilingual-uncased'

NUM_LABELS = 8

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)
model.resize_token_embeddings(len(tokenizer))

# model = pickle.load(open('model.pkl', 'rb'))

clear_output()

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))
        print (123)


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
123
classifier      Linear(in_features=768, out_features=8, bias=True)
123


In [78]:
"""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
之後也可以用來生成上傳到 Kaggle 競賽的預測結果
"""

# torch.cuda.set_device(2) # 這行加了也能跑


def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
    jindu = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
            
            jindu += BATCH_SIZE
            if jindu%1000==0:
                print(jindu, '-----')
    
    if compute_acc:
        acc = correct / total
        print(acc)
        return predictions, acc
    return predictions
    
# # 讓模型跑在 GPU 上並取得訓練集的分類準確率
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print("device:", device)
# model = model.to(device)
# test, acc = get_predictions(model, trainloader, compute_acc=True)
# print("classification acc:", acc)



In [61]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
     
model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")


整個分類模型的參數量：168221192
線性分類器的參數量：6152



In [ ]:
%%time

# torch.cuda.set_device(2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ## specify the GPU id's, GPU id's start from 0.
# model= nn.DataParallel(model,device_ids = [0, 1])
model.to(device)


! nvidia-smi

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

EPOCHS = 6  # 幸運數字
jindu = 0
for epoch in range(EPOCHS):
    running_loss = 0.0
    for data in trainloader:
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]
        # 將參數梯度歸零
        
        optimizer.zero_grad()
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)
        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()

        # 紀錄當前 batch loss
        running_loss += loss.item()
        
        jindu += BATCH_SIZE
        if jindu%1000==0:
            print (jindu)

    # 計算分類準確率
    _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))
    
    pickle.dump(model, open('model_emoji_multilingual_{}.pkl'.format(epoch), 'wb'))
    

Tue Dec 21 22:10:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 107...  Off  | 00000000:03:00.0 Off |                  N/A |
|  0%   40C    P8     7W / 180W |      2MiB /  8119MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:0A:00.0 Off |                  N/A |
|  0%   

In [ ]:
import pickle

pickle.dump(model, open('model.pkl', 'wb'))

# 對新樣本做推論

In [ ]:
%%time
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
testset = Dataset("test", tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=4, 
                        collate_fn=create_mini_batch)

# 用分類模型預測測試集
predictions = get_predictions(model, testloader)

# 用來將預測的 label id 轉回 label 文字
index_map = {v: k for k, v in testset.label_map.items()}

# 生成 Kaggle 繳交檔案
df = pd.DataFrame({"emotion": predictions.tolist()})
df['emotion'] = df.Category.apply(lambda x: index_map[x])
df_pred = pd.concat([testset.df.loc[:, ["ID"]], 
                          df.loc[:, 'Category']], axis=1)
df_pred.to_csv('bert_submission.csv', index=False)
df_pred.head()